In [1]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1. Import Dependencies

In [2]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium import Env

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 3. Building an Environment

# 6. Save Model

In [3]:

# Create custom environment
class TheBindingAI(gym.Env):
  metadata = {'render.modes': ['human']}

  def __init__(self):
    # make bigger, same and less
    self.action_space = spaces.Discrete(3)
    self.observation_space = spaces.Box(low = 0, high = 100, shape=(1,))
    self.state = 38 + random.randint(-3,3)
    self.shower_length = 60

  def step(self, action):
    # Apply temp adj
    self.state += action-1

    #decrease shower time/Periode time
    self.shower_length -= 1

    #calculate reward ==> Score
    if self.state >= 37 and self.state <39:
      reward = 1
    else:
      reward = -1
    
    # stops the learning episode
    if self.shower_length <= 0:
      # episode ends naturally
      terminated = True
    else: 
       terminated = False

    #epiode ends prematurely // Time limit
    truncated = False
    info = {}
    return self.state, reward, terminated, truncated, info
  
  
  def reset(self, seed=None, options=None):
    # Seed the environment if a seed is provided
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)

    # Reset the environment state
    self.state = np.array([38 + random.randint(-3, 3)]).astype(float)
    self.shower_length = 60

    # Return observation and optional info
    return self.state, {}
  
  def render(self, mode='human'):
    #none
    pass
  
  def close (self):
    pass


In [4]:
env=TheBindingAI()

In [5]:
env.observation_space.sample()

array([90.463104], dtype=float32)

In [6]:
env.reset()

(array([37.]), {})

# 4. Test Environment

In [7]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-16
Episode:2 Score:-46
Episode:3 Score:-20
Episode:4 Score:-48
Episode:5 Score:-56


In [8]:
env.close()

# 5. Train Model

In [9]:
log_path = os.path.join('Training', 'Logs')
model = PPO("MlpPolicy", env, learning_rate=(0.0005), verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [10]:
model.learn(total_timesteps=200000)

Logging to Training\Logs\PPO_7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -42.1    |
| time/              |          |
|    fps             | 1364     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -39.4       |
| time/                   |             |
|    fps                  | 931         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011272967 |
|    clip_fraction        | 0.0815      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.000396    |

# 6. Save Model

In [11]:
model_path = os.path.join('Training', 'Saved Models', 'TheBindingOfIsaac_PPO')

In [12]:
model.save(model_path)

In [13]:
#del model

In [14]:
#model = PPO.load(model_path, env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=20, render=False)

c:\Users\Masarum\.pyenv\pyenv-win\versions\3.12.7\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(-24.0, 54.99090833947008)